Michael Russo

661624457

Project: Introduction to Machine Learning Applications Section 2 Fall 2020

Allstate Kaggle Competition

Analysis for this code is peformed in the Kaggle online jupiter notebooks enviroment.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import given Kaggle Test and Train data sets and sample sumbission file three times.

In [ ]:
train_data = pd.read_csv("../input/allstate-claims-severity/train.csv")
test_data = pd.read_csv("../input/allstate-claims-severity/test.csv")
sample_submission = pd.read_csv("../input/allstate-claims-severity/sample_submission.csv")
sample_submission2 = sample_submission
sample_submission3= sample_submission

In [ ]:
print("Train data dimensions: ", train_data.shape)
print("Test data dimensions: ", test_data.shape)

In [ ]:
#train_data.drop('id',axis=1,inplace=True)
train_data.head()

In [ ]:
test_data.head()

In [ ]:
table1 = train_data.describe() # describe the value range, standard deviation and averages.
table1


In [ ]:

#table1.to_csv('calls.csv') # export for report


In [ ]:
#train_data.skew()

Select continious features for analysis

In [ ]:
contFeatureslist = [] # to investigate continuous data features, matrix with just continuous varaibel columns and loss 
for colName,x in train_data.iloc[1,:].iteritems():
    if(not str(x).isalpha()):
        contFeatureslist.append(colName)

print(contFeatureslist)

#contFeatureslist = ['id', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14', 'loss']

In [ ]:
contFeatureslist.remove("id")
contFeatureslist.remove("loss")

In [ ]:

train_data_simple=train_data[contFeatureslist] # investigate continous data 
#data=pd.melt(df))
train_data_simple.head()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.figure(figsize=(13,9)) # plot contious data as box plots to investigate skew, avarage, and range
sns.boxplot(x="variable", y="value", data=pd.melt(train_data_simple))


plt.figure(figsize=(13,9)) # plot contious data as violin plots to better investigate skew, avarage, and range
sns.violinplot(x="variable", y="value", data=pd.melt(train_data_simple))

In [ ]:
train_data[contFeatureslist].hist(bins=50, figsize=(16,12)) # further investigate distribution of continous data

In [ ]:
contFeatureslist.append("loss")

correlationMatrix = train_data[contFeatureslist].corr().abs() # investigate continous data coorelation between varables

plt.subplots(figsize=(13, 9))
sns.heatmap(correlationMatrix,annot=True)

# Mask unimportant features
sns.heatmap(correlationMatrix, mask=correlationMatrix < 1, cbar=False)
plt.show()

In [ ]:
s = correlationMatrix.unstack()  # list continuous varibles with most correlation

so = s.sort_values(kind="quicksort",ascending=False)

print("Top 5 most correlated continuous feature pairs: \n")
print(so[so != 1.0][0:10])


Now that we have the top 5 most correlated continuous feature pairs, we can look to remove features if needed.

In [ ]:
plt.figure(figsize=(13,9)) # investigate loss dependent variable - seen that skew is high and positive
#sns.distplot(train_data["loss"])
sns.boxplot(train_data["loss"])


Here, we can see loss is highly right skewed data. This happened because there are many outliers in the data that we can see from box plot.

logarithm on loss function Log(1+x) - This mathematical function helps user to calculate natural logarithmic value of x+1 where x belongs to all the input array elements.
It is used to get a normal distribution so that we can train model using target feature as log of loss. This way we don't have to remove outliers and algorithems can predict better.

Figure below shows skew correction.

In [ ]:
train_data["logloss"] = np.log1p(train_data["loss"]) # normalize the loss dependent varable so that model can predict better

In [ ]:
plt.figure(figsize=(13,9)) #Plot shows that skew is corrected to a large extent
#sns.distplot(np.log1p(train_data["loss"]))
sns.boxplot(train_data["logloss"])

Next we analize the Catagorial columns, Select Catagorial features for analysis with similar method as above

In [ ]:
catCount = sum(str(x).isalpha() for x in train_data.iloc[1,:]) # count number of catagorial varables
print("Number of categories: ",catCount)

In [ ]:
catFeatureslist = [] # to investigate catagorial data features, matrix with just ccatagorial varaibel columns and loss 
for colName,x in train_data.iloc[1,:].iteritems():
    if(str(x).isalpha()):
        catFeatureslist.append(colName)
        
#catFeatureslist

In [ ]:
print(train_data[catFeatureslist].apply(pd.Series.nunique)) # shows the numeber of unique catagorial values for each catagorial variables

sklearn.preprocessing import LabelEncoder is used to transform non-numerical labels to numerical labels so that model can incorperate them.

In [ ]:
from sklearn.preprocessing import LabelEncoder # trasform non-numerical labels to numerical labels so that model can incorperate them.
for cf1 in catFeatureslist:
    le = LabelEncoder()
    le.fit(train_data[cf1].unique())
    train_data[cf1] = le.transform(train_data[cf1])
    le.fit(test_data[cf1].unique())
    test_data[cf1] = le.transform(test_data[cf1])

In [ ]:
train_data.head() # view test data with new labels

In [ ]:
filter2 = list((train_data[catFeatureslist].apply(pd.Series.nunique) == 2)) # print Top 5 most correlated categorical feature pairs
catFeatures2List = [i for (i, v) in zip(catFeatureslist, filter2) if v]
catFeatures2List.append("loss")

corrCatMatrix = train_data[catFeaturesG2List].corr().abs()

s = corrCatMatrix.unstack()
so2= s.sort_values(kind="quicksort",ascending=False)

print("Top 5 most correlated categorical feature pairs: \n")
print(sortedSeries[so2 != 1.0][0:10])

Now that we have the top 5 most correlated categorical feature pairs, we can look to remove features if needed.

****Build Model****

Three models will be investigated CatBoost , XGBoost, and LightGBM. 


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor


Build Model Featuring LightGBM algorithm 

In [ ]:
X = train_data.drop(['loss',"logloss"], axis=1)
y = train_data["logloss"]

In [ ]:
X_train,X_val,y_train, y_val = train_test_split(X,y,test_size=0.1)

In [ ]:
LGBMR = LGBMRegressor(n_estimators=300, learning_rate=0.1, random_state=123)
LGBMR.fit(X_train,y_train)
LGBMRpred = LGBMR.predict(X_val)

print(mean_absolute_error(LGBMRpred,y_val))

In [ ]:
LGBMR_feature_imp = pd.DataFrame(sorted(zip(LGBMR.feature_importances_,X.columns)), columns=['Value','Feature'])

fig,ax = plt.subplots(1,1,figsize=(20,30))
sns.barplot(x="Value", y="Feature", data=LGBMR_feature_imp.sort_values(by="Value", ascending=False), ax=ax)
plt.title('LightGBM Features Importance')

In [ ]:

test_predictions = LGBMR.predict(test_data)

predictions=np.exp(test_predictions)-1
sample_submission['loss']=predictions

LGBMRResult = sample_submission.loc[:, ['id','loss']]

LGBMRResult.to_csv('Result1.csv', index=False)
LGBMRResult.head()

Build Model Featuring XGBoost algorithm 

In [ ]:
X = train_data.drop(['loss',"logloss"], axis=1)
y = train_data["logloss"]
X_train,X_val,y_train, y_val = train_test_split(X,y,test_size=0.1)

In [ ]:
XGBR = XGBRegressor(n_estimators=300, learning_rate=0.1, random_state=123)
XGBR.fit(X_train,y_train)
XGBRpred = XGBR.predict(X_val)

print(mean_absolute_error(XGBRpred,y_val))

In [ ]:
XGBR_feature_imp = pd.DataFrame(sorted(zip(XGBR.feature_importances_,X.columns)), columns=['Value','Feature'])

fig,ax = plt.subplots(1,1,figsize=(20,30))
sns.barplot(x="Value", y="Feature", data=XGBR_feature_imp.sort_values(by="Value", ascending=False), ax=ax)
plt.title('XGBRegressor Features Importance')

In [ ]:
test_predictions2 = XGBR.predict(test_data)

predictions2=np.exp(test_predictions2)-1
sample_submission2['loss']=predictions2


XGBRResult = sample_submission2.loc[:, ['id','loss']]

XGBRResult.to_csv('Result2.csv', index=False)
XGBRResult.head()

Build Model Featuring Catboost algorithm 

In [ ]:
X = train_data.drop(['loss',"logloss"], axis=1)
y = train_data["logloss"]
X_train,X_val,y_train, y_val = train_test_split(X,y,test_size=0.1)

In [ ]:
CatB = CatBoostRegressor(n_estimators=300, learning_rate=0.1, random_state=123)
CatB.fit(X_train,y_train)
CatBpred = CatB.predict(X_val)

print(mean_absolute_error(CatBpred,y_val))

In [ ]:
CatB_feature_imp = pd.DataFrame(sorted(zip(CatB.feature_importances_,X.columns)), columns=['Value','Feature'])

fig,ax = plt.subplots(1,1,figsize=(20,30))
sns.barplot(x="Value", y="Feature", data=CatB_feature_imp.sort_values(by="Value", ascending=False), ax=ax)
plt.title('CatBoostRegressorr Features Importance')

In [ ]:
test_predictions3 = CatB.predict(test_data)

predictions3=np.exp(test_predictions3)-1
sample_submission3['loss']=predictions3


CatBResult = sample_submission3.loc[:, ['id','loss']]

CatBResult.to_csv('Result3.csv', index=False)
CatBResult.head()